# Data Acquisition Code

<br>
This code includes the necessary comand to acquire the data in addition to some helper function for the image analysis


<br>

## A - Data Acquisition


<br>

### Importing Modules


<br>

In [1]:
import requests
import json
import time
import itertools
import wget
import os
import png
import pickle
import numpy as np
import tmdbsimple as tmdb
import imdb
import time
import sys
import urllib
from PIL import Image

import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import precision_recall_curve
import scipy

sns.set_style('white')
import tensorflow as tf
import pandas as pd
import keras
from imdb import IMDb
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as k
from keras.models import load_model

Using TensorFlow backend.


### Importing ID File and Genres List

<br>

In [ ]:
# Importing Data file
idsdf = pd.read_csv('randomID2.csv').iloc[9001:]

# Importing Genres List
gen_list = pd.read_csv('genres_TMDB.csv')

# Printing a sample data
idsdf.head()

### Acquiring from TMDB:


<br>

#### 1- Specifying the IDs


<br>

In [ ]:
ids = idsdf.tmdb_id.values

#### 2- Requesting the data

<br>

In [ ]:
# Accessing the TMDB
tmdb.API_KEY = '95b208af4c92eb1ddd5c831f91c84e15'

# Initiate the data Lists
t_tmdb  = []    # title
b_tmdb  = []    # budget
o_tmdb  = []    # overview
rv_tmdb = []    # revenue
k_tmdb  = []    # keywords
rt_tmdb = []    # runtime
p_tmdb  = []    # poster path
g_tmdb  = []    # Genres

# initiate counters
i = 0

for i in range(5):
    
    # Specify the movie id
    movie = ids[i]
    
    # get the movie 
    mv = tmdb.Movies(movie)
    response = mv.info()
    
    # Append to the data 
    t_tmdb.append(mv.title.encode('utf-8' , 'strict'))
    b_tmdb.append(float(mv.budget))
    o_tmdb.append(mv.overview.encode('utf-8' , 'strict'))
    rv_tmdb.append(float(mv.revenue))
    k_tmdb.append(mv.keywords)
    rt_tmdb.append(float(mv.runtime))
    p_tmdb.append(mv.poster_path.encode('utf-8' , 'strict'))
    g_tmdb.append(mv.genres)
    
    # Pause
    time.sleep(1)
    
    
    
    # Update Results
    sys.stdout.flush()
    sys.stdout.write("\r Step: {}  id {}".format(i, movie))
        


#### 3-Getting the Poster

<br>

In [ ]:
# Initial url with configuration
url1 = 'https://image.tmdb.org/t/p/w500'

# Initiate the posters list
posters = []

# Iterrate for the database
for i in range(20):
    
    # getting the movie
    url2 = p_tmdb[i]
    
    # composing the url
    url = url1 + url2
    
    filename = './posters/' + str(ids[i]) + p_tmdb[i][-4:]
    
    f = open(filename , 'wb' )
    f.write(urllib.urlopen(url).read())
    f.close()
    
    time.sleep(5)
    
    sys.stdout.flush()
    sys.stdout.write("\r Step: {} , id {}".format(i , ids[i]))

### Acquiring From IMDB :


<br>

In [ ]:
# Create an IMDB object
ia = IMDb()

# Initiate the director List
d_imdb = []
p_imdb = []

# initiate counters
i, idm = 0, 0

for i in range(20):
    
    # Specify the movie id
    movie = idsdf.imdb_id.values[i][2:]
    
    # get the movie 
    mv = ia.get_movie(movie)
        
    # Get the director object
    direc_raw = mv['director']
    
    # Get the director name
    name = str(direc_raw[0]['name'])
    
    try:
    
        # Get the plot
        plot = mv['plot']
    
        # Append to the list
        p_imdb.append(plot[0].encode('utf-8' , 'strict'))
    
        # Append to the data
        d_imdb.append(name)
        
    except KeyError as e:
        
        p_imdb.append('e')

    
    # Pause
    time.sleep(1)
    
    # Update Results
    sys.stdout.flush()
    sys.stdout.write("\r Step: {}  id {}".format(i, movie))



<br>
### Constructing the Labels Array


<br>
An array that includes a binary flag for each of the genres 

<br>

In [ ]:

# Acquire the shape of the ID array
shape_ids = idsdf.shape

# Acquire the shape of the Genres Array
shape_genre = gen_list.shape

# Intitate a multilabel array
Genres = np.zeros((shape_ids[0] , shape_genre[0]))

# Iterating for each movie
for i in range(20):
    
    # Extract the list of genres dictionaries
    genr = g_tmdb[i]
    
    # iterrate over each dictionary of the dictionaries list
    for j in range(len(genr)):
        
        # Extract the movie genre name
        gename = genr[j]['name'].encode('utf-8' , 'strict')
        
        # Compare it to the list
        for k in range(shape_genre[0]):
            
            # Change labe if there is a match
            if gename == gen_list.iloc[k,1]:
                
                Genres[i,k] = 1




<br>
## B - Helper Functions

<br>

### Handling Pixel Information

<br>

In [ ]:
# Open The image
sample = Image.open('genre_header.jpg')

# Transform it into a pixel Object
pix = sample.load()

# Determine Size
sz = sample.size

# Convert image to an array
sample_ar = sample.convert('RGB')

# Construct a numpy array
sample_array = np.array(sample_ar)

# Vectorize in RGB
vec_sample = sample_array.ravel()



- In this case the image vector will include the three colors. To handle features, we need to transform to Grayscale
- In order to get the information of each color, we will do the following


<br>

#### 1 - Red

In [ ]:
# Extract Red
arr_red = sample_array[:,:,0]

# Vectorize
vec_red = arr_red.ravel()

# Calculate Average
mean_red = np.mean(vec_red)

# Claculate the variance
var_red = vec_red.var()




<br>
#### 2 - Green

<br>

In [ ]:
# Extract Green
arr_green = sample_array[:,:,1]

# Vectorize
vec_green = arr_green.ravel()

# Calculate Average
mean_green = np.mean(vec_green)

# Calculate variance
var_green = vec_green.var()




<br>
#### 2 -Blue

<br>

In [ ]:
# Extract Blue
arr_blue = sample_array[:,:,2]

# Vectorize
vec_blue = arr_blue.ravel()

# Calculate Average
mean_blue = np.mean(vec_blue)

# Calculate Variance
var_blue = vec_blue.var()



<br>

### Convert to grayscale and vectorize (As a feature)

<br>

In [ ]:
# Convert to grayscale
img_gray = sample.convert("1")

# Construct a numpy array
sample_gray = np.array(img_gray)

# Vectorize in RGB
vec_gray = sample_gray.ravel()

In [ ]:

# Check if All three colors were condensed into in a monochrome value
float(vec_sample.shape[0]) / float(vec_gray.shape[0])